In [1]:
import pandas as pd
cdf = pd.read_parquet("abs_type_approvals.parquet")
cdf.info()

StatementMeta(55aa7293-6613-4fd9-bd05-89ce79d2509a, 0, 6, Finished, Available)

FileNotFoundError: [Errno 2] No such file or directory: 'abs_type_approvals.parquet'

In [42]:
cdf['category'].unique()

array([None, 'Design Assessed', 'Type Approved', 'EC Type Examined',
       'Wheelmarked'], dtype=object)

In [3]:
import base64, hashlib
def hash(x):
  return "id-" + base64.urlsafe_b64encode(hashlib.sha1(x).digest()).decode("utf-8") 

In [4]:
import json
import requests
from pprint import pprint

In [5]:
endpoint = 'https://gptkb-r5e7unxol46om.search.windows.net/'
api_version = '?api-version=2019-05-06'
headers = {'Content-Type': 'application/json',
        'api-key': 'kxgH5lLlChcWrsiGObjpjIrvwednYcUOaFgiZ12NXYAzSeBrN9g8' }

In [11]:
with open('typeapprovalindex.json') as json_file:
    index_schema = json.load(json_file)

In [7]:
index_name="typeapprovalindex"

In [8]:
url = endpoint + "indexes/" + index_name + api_version
response  = requests.delete(url, headers=headers)
print(response)

<Response [204]>


In [ ]:
url = endpoint + "indexes" + api_version
response  = requests.post(url, headers=headers, json=index_schema)
index = response.json()
pprint(index)

In [ ]:
url = endpoint + "indexes" + api_version + "&$select=name"
response  = requests.get(url, headers=headers)
index_list = response.json()
pprint(index_list)

In [ ]:
def ingest_docs(doc_df):
    doc_df = doc_df.fillna('')
    documents = {"value": doc_df.to_dict(orient="records")}
    url = endpoint + "indexes/" + index_name +"/docs/index" + api_version
    response  = requests.post(url, headers=headers, json=documents)
    response_df = pd.DataFrame.from_dict(response.json())
    return response_df

In [ ]:
cdf.head()

In [16]:
# id = modelCertId
# content = product + description + absRules + ratings
# category = category
# sourcepage = companyname + address1 + country
# sourcefile = certificateNumber
# issuedate = issueDate
# expirydate = expirydate

In [ ]:
search_df = cdf[['modelcertId', 'product', 'description', 'absRules', 'ratings', 'category', 'companyname', 'address1', 'country', 'certificateNumber', 'issueDate', 'expirydate']]
content_cols = ['product', 'description', 'absRules', 'ratings']
company_cols = ['companyname', 'address1', 'country']
search_df['content'] = search_df[content_cols].apply(lambda row: ' || '.join(row.values.astype(str)), axis=1)
search_df['company'] = search_df[company_cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
search_df = search_df.drop(content_cols, axis=1)
search_df = search_df.drop(company_cols, axis=1)
search_df = search_df.rename(columns={'modelcertId': 'id', 'company': 'sourcepage', 'certificateNumber': 'sourcefile', 'issueDate': 'issuedate'})
search_df = search_df.dropna(subset=['issuedate', 'expirydate'])
search_df['issuedate'] = search_df['issuedate'].map(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
search_df['expirydate'] = search_df['expirydate'].map(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
search_df['id'] = search_df['id'].astype(str)
search_df.head()

In [37]:
search_df["@search.action"] = "upload"

In [ ]:
search_df.info()

In [39]:
import numpy as np
partitions = 1000
data_splits = np.array_split(search_df, partitions)
for data_split in data_splits: 
    doc_df = data_split.fillna('')
    documents = {"value": doc_df.to_dict(orient="records")}
    url = endpoint + "indexes/" + index_name +"/docs/index" + api_version
    response  = requests.post(url, headers=headers, json=documents)
    #print(response.json())
    #break
    

In [40]:
search_df.to_parquet("typeapprovaldata.parquet")